In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    CSVLogger,
    LearningRateScheduler,
)

# --- 1) Config ---
data_dir = '/kaggle/input/preprocessed/mtc-aic3_dataset_preprocessed'  # Root of preprocessed dataset
output_dir = './models'
os.makedirs(output_dir, exist_ok=True)

# --- 2) Data Loading Function ---
def load_dataset(csv_name, split):
    """Load EEG data and labels from CSV files"""
    # Read metadata CSV
    df = pd.read_csv(os.path.join(data_dir, csv_name))
    mi_df = df[df['task'] == 'MI']  # Filter for MI tasks
    
    X, y = [], []
    session_cache = {}  # Cache session data to avoid repeated reads
    
    # Known EEG channels in the dataset
    eeg_channels = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
    
    for _, row in mi_df.iterrows():
        # Construct EEG data path
        eeg_path = os.path.join(
            data_dir, 'MI', split,
            row['subject_id'], str(row['trial_session']), 'EEGdata.csv'
        )
        
        # Cache session data
        if eeg_path not in session_cache:
            try:
                session_data = pd.read_csv(eeg_path)
                
                # Verify and select only EEG channels
                available_cols = session_data.columns.tolist()
                keep_cols = [col for col in available_cols if col in eeg_channels or col == 'Time']
                session_data = session_data[keep_cols]
                
                # Drop Time column if exists
                if 'Time' in session_data.columns:
                    session_data = session_data.drop(columns=['Time'])
                
                # Verify we have exactly 8 channels
                if len(session_data.columns) != 8:
                    raise ValueError(f"Expected 8 channels, found {len(session_data.columns)} in {eeg_path}")
                
                # Reshape to (trials, time, channels)
                n_trials = 10
                n_samples = 375
                total_samples = n_trials * n_samples
                
                # Verify data size
                if len(session_data) != total_samples:
                    raise ValueError(f"Unexpected data size in {eeg_path}: "
                                    f"Expected {total_samples} rows, got {len(session_data)}")
                
                # Reshape and store in cache
                session_cache[eeg_path] = session_data.values.reshape(n_trials, n_samples, len(eeg_channels))
            except Exception as e:
                print(f"Error loading {eeg_path}: {str(e)}")
                raise
        
        # Extract trial (375 samples, 8 channels)
        trial_idx = row['trial'] - 1
        trial_data = session_cache[eeg_path][trial_idx]
        X.append(trial_data)
        y.append(row['label'])
    
    return np.array(X), np.array(y)

# --- 3) Data Cleaning Function ---
def clean_data(X):
    """Remove NaNs, Infs, and handle extreme values"""
    print("Cleaning data...")
    nan_count = np.isnan(X).sum()
    inf_count = np.isinf(X).sum()
    
    if nan_count > 0 or inf_count > 0:
        print(f"Cleaning: Found {nan_count} NaNs and {inf_count} Infs")
    
    # Replace NaNs and Infs with channel mean
    for i in range(X.shape[0]):  # For each trial
        for j in range(X.shape[2]):  # For each channel
            channel_data = X[i, :, j]
            
            # Find and replace NaNs/Infs
            mask = np.isnan(channel_data) | np.isinf(channel_data)
            if np.any(mask):
                mean_val = np.nanmean(channel_data[~mask])
                if np.isnan(mean_val) or np.isinf(mean_val):
                    mean_val = 0.0  # Fallback if mean is still problematic
                channel_data[mask] = mean_val
                X[i, :, j] = channel_data
            
            # Clip extreme values (±1000μV)
            np.clip(channel_data, -1000, 1000, out=channel_data)
    
    # Verify cleaning results
    post_nan = np.isnan(X).sum()
    post_inf = np.isinf(X).sum()
    
    if post_nan > 0 or post_inf > 0:
        print(f"Warning: Still found {post_nan} NaNs and {post_inf} Infs after cleaning")
    else:
        print("Data cleaning successful - no NaNs or Infs remaining")
    
    return X

# --- 4) Data Inspection ---
def inspect_data(X, y, name):
    """Print data statistics for quality control"""
    print(f"\n=== {name} Data Inspection ===")
    print(f"Shape: {X.shape} (trials × samples × channels)")
    print(f"Labels: {np.unique(y, return_counts=True)}")
    
    # Channel statistics
    channel_names = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
    for i, ch in enumerate(channel_names):
        chan_data = X[:, :, i]
        print(f"{ch}: Min={np.min(chan_data):.2f}μV, "
              f"Max={np.max(chan_data):.2f}μV, "
              f"Mean={np.mean(chan_data):.2f}μV, "
              f"NaNs={np.isnan(chan_data).sum()}, "
              f"Infs={np.isinf(chan_data).sum()}")

# --- 5) Load and clean data ---
print("Loading training data...")
X_train, y_train = load_dataset('train.csv', 'train')
print("Loading validation data...")
X_val, y_val = load_dataset('validation.csv', 'validation')

# Data inspection
inspect_data(X_train, y_train, "Training")
inspect_data(X_val, y_val, "Validation")

# Clean data
X_train = clean_data(X_train)
X_val = clean_data(X_val)

# Binarize labels
y_train_bin = (y_train == 'Right').astype(int)
y_val_bin = (y_val == 'Right').astype(int)

# --- 6) CSP (4 components) ---
print("Applying CSP...")
X_train_csp = X_train.transpose(0, 2, 1)  # Convert to (n, 8, 375) for CSP

# Add small noise to prevent singular matrix issues
X_train_csp += np.random.normal(0, 1e-10, X_train_csp.shape)

csp = CSP(n_components=4, log=False, norm_trace=False)
csp.fit(X_train_csp, y_train_bin)
W = csp.filters_[:4]

def apply_csp(X):
    """Apply CSP filters to EEG data"""
    X_csp = X.transpose(0, 2, 1)  # Convert to (n, 8, 375)
    return np.stack([W.dot(ep) for ep in X_csp], axis=0)

print("Transforming training data with CSP...")
Xtr = apply_csp(X_train).astype('float32')  # (n, 4, 375)
Xtr = Xtr.transpose(0, 2, 1)               # (n, 375, 4) for models
print("Transforming validation data with CSP...")
Xvl = apply_csp(X_val).astype('float32')    # (n, 4, 375)
Xvl = Xvl.transpose(0, 2, 1)               # (n, 375, 4)

# --- 7) For Model3 only: add channel axis → (n, T, F=4, 1) ---
Xtr_spec = Xtr[..., np.newaxis]
Xvl_spec = Xvl[..., np.newaxis]

# --- 8) One‑hot labels ---
ytr_oh = keras.utils.to_categorical(y_train_bin, 2)
yvl_oh = keras.utils.to_categorical(y_val_bin, 2)

# --- 9) Data‑augmentation gens ---
def aug_gen(X, y, seed=0, batch_size=32):
    n = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        # Add Gaussian noise and random scaling
        bx += rng.normal(0, 0.5, bx.shape)  # Increased noise magnitude
        bx *= rng.uniform(0.9, 1.1, bx.shape)  # Random scaling
        yield bx, by

train_gen_1d = aug_gen(Xtr, ytr_oh, seed=0, batch_size=64)
train_gen_2d = aug_gen(Xtr_spec, ytr_oh, seed=1, batch_size=64)

steps_1d = len(Xtr) // 64
steps_2d = len(Xtr_spec) // 64

# --- 10) LR schedule ---
def cosine_lr(epoch, lr_max=1e-4, epochs=200):  # Lower max LR
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 11) F1Score metric ---
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        preds = tf.cast(preds, tf.int32)
        labels = tf.cast(labels, tf.int32)

        tp = tf.reduce_sum(
            tf.cast(tf.logical_and(preds == 1, labels == 1), tf.float32)
        )
        fp = tf.reduce_sum(
            tf.cast(tf.logical_and(preds == 1, labels == 0), tf.float32)
        )
        fn = tf.reduce_sum(
            tf.cast(tf.logical_and(preds == 0, labels == 1), tf.float32)
        )

        self.tp.assign_add(tp)
        self.fp.assign_add(fp)
        self.fn.assign_add(fn)

    def result(self):
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

# --- 12) Callbacks factory ---
def get_callbacks(name):
    return [
        EarlyStopping("val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(
            os.path.join(output_dir, f"best_{name}.h5"),
            "val_f1_score", mode="max", save_best_only=True
        ),
        CSVLogger(os.path.join(output_dir, f"log_{name}.csv")),
        LearningRateScheduler(cosine_lr)
    ]

# --- 13) Model builders ---
def build_modelA(input_shape):
    m = keras.Sequential([
        layers.Input(input_shape),
        layers.Conv1D(32, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(64, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(128,5,activation="relu",padding="same"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu", 
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.7),
        layers.Dense(2, activation="softmax"),
    ])
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_modelB(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for f in [16,32,64,128,256]:
        x = layers.Conv1D(f,3,activation="relu",padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu",
                         kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.Dropout(0.5)(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model1(input_shape):
    inp = layers.Input(input_shape+(1,))
    x = layers.Concatenate()([inp, inp, inp])
    x = layers.Resizing(32,32)(x)
    base = keras.applications.ResNet50(
        include_top=False, weights="imagenet",
        input_shape=(32,32,3), pooling="avg"
    )
    base.trainable = False
    x = base(x)
    x = layers.Reshape((1, x.shape[-1]))(x)
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.MultiHeadAttention(num_heads=4,key_dim=32)(x,x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model2(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(32,3,activation="elu",padding="same")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model3(input_shape):
    inp = layers.Input(input_shape)  # (T, F, 1)
    x = inp
    for _ in range(5):
        x = layers.Conv2D(32,(3,3),activation="relu",padding="same")(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model4(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.LSTM(128)(x)
    for _ in range(4):
        x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model5(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="elu",padding="same")(x)
    x = layers.Flatten()(x)
    for _ in range(3):
        x = layers.Dense(64, activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model6(input_shape):
    C3, C4 = 0,2
    eeg_in = layers.Input(input_shape)
    c3 = layers.Lambda(lambda x: x[:,:,C3:C3+1])(eeg_in)
    c4 = layers.Lambda(lambda x: x[:,:,C4:C4+1])(eeg_in)
    def branch():
        return models.Sequential([
            layers.Conv1D(16,250,activation="relu",padding="same"),
            layers.MaxPool1D(3),
            layers.Conv1D(32,50,activation="relu",padding="same"),
            layers.GlobalAveragePooling1D()
        ])
    b3, b4 = branch()(c3), branch()(c4)
    x = layers.Concatenate()([b3,b4])
    for _ in range(4):
        x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2,activation="softmax")(x)
    m = models.Model(eeg_in,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

# --- 14) Train & eval loop ---
builders = {
    'modelA': build_modelA,
    'modelB': build_modelB,
    'model1': build_model1,
    'model2': build_model2,
    'model3': build_model3,
    'model4': build_model4,
    'model5': build_model5,
    'model6': build_model6,
}

results = {}
shape_1d = Xtr.shape[1:]      # (375, 4)
shape_2d = Xtr_spec.shape[1:] # (375, 4, 1)

print("\n=== Starting Model Training ===")
for name, build_fn in builders.items():
    print(f"\n>>> Training {name}")
    if name == 'model3':
        model = build_fn(shape_2d)
        gen   = train_gen_2d
        steps = steps_2d
        val_x = Xvl_spec
    else:
        model = build_fn(shape_1d)
        gen   = train_gen_1d
        steps = steps_1d
        val_x = Xvl

    # Add model summary
    model.summary()
    
    # Train with error handling
    try:
        history = model.fit(
            gen,
            steps_per_epoch=steps,
            validation_data=(val_x, yvl_oh),
            epochs=100,  # Reduced epochs for faster iteration
            callbacks=get_callbacks(name),
            verbose=2
        )

        # Evaluate best model
        preds = np.argmax(model.predict(val_x), axis=1)
        f1 = f1_score(y_val_bin, preds)
        print(f"{name} → val F1 = {f1:.4f}")
        print(classification_report(y_val_bin, preds, target_names=['Left','Right']))
        results[name] = (f1, model)
    except Exception as e:
        print(f"Error training {name}: {str(e)}")
        results[name] = (0.0, None)

# --- 15) Save best model ---
# Filter out failed models
successful_results = {k: v for k, v in results.items() if v[0] > 0}
if successful_results:
    best_name, (best_f1, best_model) = max(successful_results.items(), key=lambda kv: kv[1][0])
    print(f"\n=== Final best: {best_name} (F1={best_f1:.4f}) ===")
    best_model.save(os.path.join(output_dir, 'best_final.h5'))
else:
    print("\n!!! No models trained successfully !!!")

Loading training data...
Loading validation data...

=== Training Data Inspection ===
Shape: (2400, 375, 8) (trials × samples × channels)
Labels: (array(['Left', 'Right'], dtype='<U5'), array([1187, 1213]))
FZ: Min=225946.38μV, Max=456689.25μV, Mean=298502.13μV, NaNs=0, Infs=0
C3: Min=223116.38μV, Max=456487.47μV, Mean=301820.80μV, NaNs=0, Infs=0
CZ: Min=nanμV, Max=nanμV, Mean=nanμV, NaNs=30375, Infs=0
C4: Min=nanμV, Max=nanμV, Mean=nanμV, NaNs=42375, Infs=0
PZ: Min=nanμV, Max=nanμV, Mean=nanμV, NaNs=111000, Infs=0
PO7: Min=178733.77μV, Max=353583.69μV, Mean=288641.41μV, NaNs=0, Infs=0
OZ: Min=222399.95μV, Max=382140.81μV, Mean=285756.78μV, NaNs=0, Infs=0
PO8: Min=166140.80μV, Max=358059.41μV, Mean=286186.18μV, NaNs=0, Infs=0

=== Validation Data Inspection ===
Shape: (50, 375, 8) (trials × samples × channels)
Labels: (array(['Left', 'Right'], dtype='<U5'), array([28, 22]))
FZ: Min=247211.28μV, Max=325971.16μV, Mean=294708.74μV, NaNs=0, Infs=0
C3: Min=263503.75μV, Max=350721.59μV, Mean

/tmp/ipykernel_35/537930018.py:101: RuntimeWarning: Mean of empty slice
  mean_val = np.nanmean(channel_data[~mask])


Data cleaning successful - no NaNs or Infs remaining
Cleaning data...
Cleaning: Found 7125 NaNs and 0 Infs
Data cleaning successful - no NaNs or Infs remaining
Applying CSP...
Computing rank from data with rank=None
    Using tolerance 4.7e+03 (2.2e-16 eps * 8 dim * 2.6e+18  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Transforming training data with CSP...
Transforming validation data with CSP...

=== Starting Model Training ===

>>> Training modelA


2025-06-27 02:56:36.421763: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

...

 Total params: 76,066 (297.13 KB)

 Trainable params: 76,066 (297.13 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
37/37 - 11s - 293ms/step - accuracy: 0.5169 - f1_score: 0.6266 - loss: 0.6939 - val_accuracy: 0.5600 - val_f1_score: 0.0000e+00 - val_loss: 0.6922 - learning_rate: 0.0010
Epoch 2/100
37/37 - 6s - 169ms/step - accuracy: 0.5013 - f1_score: 0.5568 - loss: 0.6960 - val_accuracy: 0.4400 - val_f1_score: 0.6111 - val_loss: 0.6948 - learning_rate: 9.9994e-04
Epoch 3/100
37/37 - 5s - 142ms/step - accuracy: 0.5063 - f1_score: 0.6723 - loss: 0.6933 - val_accuracy: 0.4400 - val_f1_score: 0.6111 - val_loss: 0.6945 - learning_rate: 9.9969e-04
Epoch 4/100
37/37 - 5s - 147ms/step - accuracy: 0.4932 - f1_score: 0.2849 - loss: 0.6935 - val_accuracy: 0.5600 - val_f1_score: 0.0000e+00 - val_loss: 0.6929 - learning_rate: 9.9914e-04
Epoch 5/100
37/37 - 5s - 146ms/step - accuracy: 0.4873 - f1_score: 0.3749 - loss: 0.6933 - val_accuracy: 0.5600 - val_f1_score: 0.0000e+00 - val_loss: 0.6923 - learning_rate: 9.9815e-04
Epoch 6/100
37/37 - 5s - 136ms/step - accuracy: 0.4772 - f1_score: 0.2320 - loss:

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
